<a id="toc"></a>
# Benchmarking
***

Contents :
1. [Mono entrance slit](#mono_in)
1. [Mono exit slit](#mono_out)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '30/JUL/2024'
__changed__ = '07/AUG/2024'

In [ ]:
import sys
import numpy as np
from numpy.polynomial import Polynomial
from barc4plots.barc4plots import PlotManager
import barc4xoc.aux_pyoptix as apx
import barc4xoc.aux_shadow as asw
from array import array
from scipy.optimize import minimize
# checking we are using the correct python:
print(sys.executable)
print(sys.version)
# %matplotlib widget

In [ ]:
def plot_phase_space(beam, direction, prange, file_name=None):
    d = direction.capitalize()
    sts = get_beam_stats(beam, f"{d.casefold()}")
    fig = PlotManager(axis_x=beam[f"{d}"]*1E6, axis_y=beam[f"{d}p"]*1E6)
    fig.additional_info('', f"${d.casefold()}$ [µm]", f"${d.casefold()}_p$ [µrad]", xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3])
    fig.aesthetics(dpi=600, LaTex=True, AspectRatio=False, FontsSize=None, grid=True, nbins=None)
    fig.info_scatter(ColorScheme=9, LineStyle='.', alpha=1, s=1, edgeColors='face', monochrome=False)
    fig.plot_scatter_hist(file_name)

def plot_beam(beam, prange, file_name=None):
    # sts = get_beam_stats(beam, "x")
    # sts = get_beam_stats(beam, "y")
    fig = PlotManager(axis_x=beam["X"]*1E6, axis_y=beam["Y"]*1E6)
    fig.additional_info('', 'µm', 'µm', xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3])
    fig.aesthetics(dpi=600, LaTex=True, AspectRatio=False, FontsSize=None, grid=True, nbins=None)
    fig.info_scatter(ColorScheme=9, LineStyle='.', alpha=1, s=1, edgeColors='face', monochrome=False, showXhist=False)
    fig.plot_scatter_hist(file_name)

def plot_intensity(wft, prange, file_name=None):
    # sts = get_beam_stats(beam, "x")
    # sts = get_beam_stats(beam, "y")
    intensity = wft["wavefront"]["intensity"]
    intensity[intensity<10e-5] = np.nan
    fig = PlotManager(wft["wavefront"]["intensity"], axis_x=wft["axis"]["x"]*1E6, axis_y=wft["axis"]["y"]*1E6)
    fig.additional_info('', 'µm', 'µm', xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3])
    fig.aesthetics(dpi=None, LaTex=True, AspectRatio=False, FontsSize=None, grid=True, nbins=None)
    fig.info_2d_plot(ColorScheme=9)
    fig.plot_2d_cuts(file_name)

def calculate_range(X1, X2, Y1, Y2, mode=0, **kwargs):
    """
    Calculate the range for plotting based on the given vectors.

    Parameters:
    X1, X2, Y1, Y2: numpy arrays or lists
        The vectors containing the data.
    mode: str, optional
        The mode for calculating the range. Can be 'independent' or 'same'.
        'independent' calculates ranges independently for X and Y.
        'same' calculates a single range based on the maximum of X and Y.

    Returns:
    tuple:
        (xmin, xmax, ymin, ymax)
    """
    nsigma = kwargs.get("nsigma", 3)
    # Calculate the max absolute values for X and Y vectors
    max_abs_X = max(np.max(np.abs(X1)), np.max(np.abs(X2)))*1.05
    max_abs_Y = max(np.max(np.abs(Y1)), np.max(np.abs(Y2)))*1.05
    max_sigma_X = max(np.std(X1), np.std(X2))*nsigma
    max_sigma_Y = max(np.std(Y1), np.std(Y2))*nsigma
    if mode == 0:
        # Independent ranges for X and Y
        xmin, xmax = -max_abs_X, max_abs_X
        ymin, ymax = -max_abs_Y, max_abs_Y
    elif mode == 1:
        # Same range for both X and Y based on the max of max_abs_X and max_abs_Y
        max_abs = max(max_abs_X, max_abs_Y)
        xmin, xmax = -max_abs, max_abs
        ymin, ymax = -max_abs, max_abs
    elif mode == 2:
        xmin, xmax = -max_sigma_X, max_sigma_X
        ymin, ymax = -max_sigma_Y, max_sigma_Y
    elif mode == 3:
        max_abs = max(max_sigma_X, max_abs_Y)
        xmin, xmax = -max_abs, max_abs
        ymin, ymax = -max_abs, max_abs

    else:
        raise ValueError("Invalid mode")

    return np.asarray((xmin, xmax, ymin, ymax))

def get_beam_stats(beam, direction="both"):

    def objective_function(distance, spots, axis):
        return (spots[axis.capitalize()] + distance * spots[axis.capitalize()+"p"]).std()
    
    results = []

    if direction in ['x', 'both']:
        result_x = minimize(objective_function, 0, args=(beam, 'x'))
        results.append(result_x.x[0])
        print("Horizontal plane:")

        s = np.std(beam["X"])
        ds = np.std(beam["Xp"])
        msg = f">> RMS beam size {s*1E6:.3f} µm and divergence {ds*1E6:.3f} µrad"
        print(msg)
        print(f">> Beam focusing along X at {result_x.x[0]:.6f} m")

    if direction in ['y', 'both']:
        result_y = minimize(objective_function, 0, args=(beam, 'y'))
        results.append(result_y.x[0])

        s = np.std(beam["Y"])
        ds = np.std(beam["Yp"])
        print("Vertical plane:")
        msg = f">> RMS beam size {s*1E6:.3f} µm and divergence {ds*1E6:.3f} µrad"
        print(msg)
        print(f">> Beam focusing along Y at {result_y.x[0]:.6f} m")

    return results
    
def merge_beams(beams):

    merged_beam = {
    'energy': np.array([]),
    'intensity': np.array([]),
    'X': np.array([]),
    'Y': np.array([]),
    'Z': np.array([]),
    'Xp': np.array([]),
    'Yp': np.array([]),
    'Zp': np.array([]),
    }
    for beam in beams:
        for key in merged_beam.keys():
            merged_beam[key] = np.append(merged_beam[key], beam[key])

    return merged_beam

class SRWLRadMesh(object):
    """Radiation Mesh (Sampling)"""
    
    def __init__(self, _eStart=0, _eFin=0, _ne=1, _xStart=0, _xFin=0, _nx=1, _yStart=0, _yFin=0, _ny=1, _zStart=0):

        self.eStart = _eStart
        self.eFin = _eFin
        self.ne = _ne
        self.xStart = _xStart
        self.xFin = _xFin
        self.nx = _nx
        self.yStart = _yStart
        self.yFin = _yFin
        self.ny = _ny
        self.zStart = _zStart


def srwl_uti_read_intens_ascii(_file_path, _num_type='f'):

    sCom = '#'
    f = open(_file_path, 'r')
    lines = f.readlines()

    resMesh = SRWLRadMesh()

    curParts = lines[1].split(sCom); resMesh.eStart = float(curParts[1]) #to check
    curParts = lines[2].split(sCom); resMesh.eFin = float(curParts[1]) #to check
    curParts = lines[3].split(sCom); resMesh.ne = int(curParts[1]) #to check
    
    curParts = lines[4].split(sCom); resMesh.xStart = float(curParts[1]) #to check
    curParts = lines[5].split(sCom); resMesh.xFin = float(curParts[1]) #to check
    curParts = lines[6].split(sCom); resMesh.nx = int(curParts[1]) #to check

    curParts = lines[7].split(sCom); resMesh.yStart = float(curParts[1]) #to check
    curParts = lines[8].split(sCom); resMesh.yFin = float(curParts[1]) #to check
    curParts = lines[9].split(sCom); resMesh.ny = int(curParts[1]) #to check

    iStart = 10
    if((lines[10])[0] == sCom): iStart = 11
    
    nRows = len(lines)
    arInt = []
    for i in range(iStart, nRows):
        curLine = lines[i]
        if(len(curLine) > 0): arInt.append(float(curLine))
    f.close()
    return array(_num_type, arInt), resMesh


def read_srw_intensity_dat(file_name, bandwidth=1e-3, transmission=1, norm=True):

    image, mesh = srwl_uti_read_intens_ascii(file_name)
    image = np.reshape(image, (mesh.ny, mesh.nx))
    dx = (mesh.xFin - mesh.xStart)/mesh.nx * 1E3
    dy = (mesh.yFin - mesh.yStart)/mesh.ny * 1E3

    image = image*dx*dy*bandwidth*transmission/(1e-3)
    n = 1
    if norm is not False:
        if norm is True:
            image /= np.amax(image)
            n = np.amax(image)
        else:
            image /= norm
            n = norm

    x = np.linspace(mesh.xStart, mesh.xFin, mesh.nx)
    y = np.linspace(mesh.yStart, mesh.yFin, mesh.ny)

    wftDict = {
        "axis": {
            "x": x,
            "y": y,
            },
        "wavefront": {
            "intensity":image,
            "phase": None,
            }
        }
    return wftDict

<a id="mono_in"></a>
## Mono entrance slit
[Back to the top](#toc)

In [ ]:
px_mes = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_mono_entrance_slit.csv")
sw_mes = asw.read_shadow_beam_from_csv("./results/shadow_G450_mono_entrance_slit.csv")

prangex = calculate_range(px_mes["X"], sw_mes["X"], px_mes["Xp"], sw_mes["Xp"], 0, nsigma=4)*1E6
prangey = calculate_range(px_mes["Y"], sw_mes["Y"], px_mes["Yp"], sw_mes["Yp"], 0, nsigma=4)*1E6

### PyOptiX

In [ ]:
plot_phase_space(px_mes, 'x', prangex, file_name="pyoptix_G450_mono_entrance_slit_xdx")
plot_phase_space(px_mes, 'y', prangey, file_name="pyoptix_G450_mono_entrance_slit_ydy")

### Shadow

In [ ]:
plot_phase_space(sw_mes, 'x', prangex, file_name="shadow_G450_mono_entrance_slit_xdx")
plot_phase_space(sw_mes, 'y', prangey, file_name="shadow_G450_mono_entrance_slit_ydy")

<a id="mono_out"></a>
## Mono exit slit
[Back to the top](#toc)

In [ ]:
px_mes = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_mono_exit_slit.csv")
sw_mes = asw.read_shadow_beam_from_csv("./results/shadow_G450_mono_exit_slit.csv")

prangex = calculate_range(px_mes["X"], sw_mes["X"], px_mes["Xp"], sw_mes["Xp"], 0, nsigma=4)*1E6
prangey = calculate_range(px_mes["Y"], sw_mes["Y"], px_mes["Yp"], sw_mes["Yp"], 0, nsigma=4)*1E6

### PyOptiX

In [ ]:
plot_phase_space(px_mes, 'x', prangex, file_name="pyoptix_G450_mono_exit_slit_xdx")
plot_phase_space(px_mes, 'y', prangey, file_name="pyoptix_G450_mono_exit_slit_ydy")

### Shadow

In [ ]:
plot_phase_space(sw_mes, 'x', prangex, file_name="shadow_G450_mono_exit_slit_xdx")
plot_phase_space(sw_mes, 'y', prangey, file_name="shadow_G450_mono_exit_slit_ydy")

<a id="STXM"></a>
## STXM
[Back to the top](#toc)

In [ ]:
px_mes = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_stxm.csv")
sw_mes = asw.read_shadow_beam_from_csv("./results/shadow_G450_stxm.csv")

prangex = calculate_range(px_mes["X"], sw_mes["X"], px_mes["Xp"], sw_mes["Xp"], 0, nsigma=4)*1E6
prangey = calculate_range(px_mes["Y"], sw_mes["Y"], px_mes["Yp"], sw_mes["Yp"], 0, nsigma=4)*1E6

### PyOptiX

In [ ]:
plot_phase_space(px_mes, 'x', prangex, file_name="pyoptix_G450_stxm_xdx")
plot_phase_space(px_mes, 'y', prangey, file_name="pyoptix_G450_stxm_ydy")

### Shadow

In [ ]:
plot_phase_space(sw_mes, 'x', prangex, file_name="shadow_G450_stxm_xdx")
plot_phase_space(sw_mes, 'y', prangey, file_name="shadow_G450_stxm_ydy")

<a id="peem"></a>
## PEEM
[Back to the top](#toc)

In [ ]:
px_mes = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_peem.csv")
sw_mes = asw.read_shadow_beam_from_csv("./results/shadow_G450_peem.csv")

prangex = calculate_range(px_mes["X"], sw_mes["X"], px_mes["Xp"], sw_mes["Xp"], 0, nsigma=4)*1E6
prangey = calculate_range(px_mes["Y"], sw_mes["Y"], px_mes["Yp"], sw_mes["Yp"], 0, nsigma=4)*1E6

### PyOptiX

In [ ]:
plot_phase_space(px_mes, 'x', prangex, file_name="pyoptix_G450_peem_xdx")
plot_phase_space(px_mes, 'y', prangey, file_name="pyoptix_G450_peem_ydy")

### Shadow

In [ ]:
plot_phase_space(sw_mes, 'x', prangex, file_name="shadow_G450_peem_xdx")
plot_phase_space(sw_mes, 'y', prangey, file_name="shadow_G450_peem_ydy")

## G450 l/mm - Resolving power - $\Delta E/E=7'500$
[Back to the top](#toc)

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP7500_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP7500_n.csv")
prange = [-520,520,-220, 220]#calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=5)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_G450_RP7500")

In [ ]:
Eo = asw.read_shadow_beam_from_csv("./results/shadow_G450_RP7500.csv")
plot_beam(Eo, prange, file_name="shadow_G450_RP7500")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP7500_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP7500_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_G450_RP7500")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP7500_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP7500_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_M1bis_G450_RP7500")

## G450 l/mm - Resolving power - $\Delta E/E=10'000$
[Back to the top](#toc)

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP10000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP10000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_G450_RP10000")

In [ ]:
Eo = asw.read_shadow_beam_from_csv("./results/shadow_G450_RP10000.csv")
plot_beam(Eo, prange, file_name="shadow_G450_RP10000")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP10000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP10000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_G450_RP10000")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP10000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP10000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_M1bis_G450_RP10000")

## G450 l/mm - Resolving power - $\Delta E/E=15'000$
[Back to the top](#toc)

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP15000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP15000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_G450_RP15000")

In [ ]:
Eo = asw.read_shadow_beam_from_csv("./results/shadow_G450_RP15000.csv")
plot_beam(Eo, prange, file_name="shadow_G450_RP15000")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP15000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP15000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_G450_RP15000")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP15000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP15000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_M1bis_G450_RP15000")

## G450 l/mm - Resolving power - $\Delta E/E=20'000$
[Back to the top](#toc)

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP20000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_G450_RP20000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_G450_RP20000")

In [ ]:
Eo = asw.read_shadow_beam_from_csv("./results/shadow_G450_RP20000.csv")
plot_beam(Eo, prange, file_name="shadow_G450_RP20000")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP20000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_G450_RP20000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_G450_RP20000")

In [ ]:
Eo = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_mono_exit_slit.csv")
Ep = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP20000_p.csv")
En = apx.read_pyoptix_beam_from_csv("./results/pyoptix_S2_M1bis_G450_RP20000_n.csv")
# prange = calculate_range(En["X"], Ep["X"], En["Y"], Ep["Y"], 0, nsigma=4)*1E6
beam = merge_beams([Eo, Ep, En])
plot_beam(beam, prange, file_name="pyoptix_S2_M1bis_G450_RP20000")